# SAE Project - Session 4: System Integration and Optimization

## Session Objectives (4 hours)

1. Complete system integration with simulation
2. Implement real-time visualization
3. Optimize cycle time and throughput
4. Add Industry 4.0 monitoring and data logging
5. Final testing and performance evaluation

---

## Part 4.1: Complete System Architecture

### Industry 4.0 Quality Control Cell

```
┌─────────────────────────────────────────────────────────┐
│                  CONTROL LAYER                          │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐ │
│  │Motion Planner│  │Vision System │  │Data Analytics│ │
│  └──────────────┘  └──────────────┘  └──────────────┘ │
└─────────────────────────────────────────────────────────┘
                          ↓
┌─────────────────────────────────────────────────────────┐
│              EXECUTION LAYER                            │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐ │
│  │OpenManip-X   │  │Vision Camera │  │Gripper       │ │
│  │(Robot)       │  │& Sensors     │  │Actuator      │ │
│  └──────────────┘  └──────────────┘  └──────────────┘ │
└─────────────────────────────────────────────────────────┘
                          ↓
┌─────────────────────────────────────────────────────────┐
│              PHYSICAL LAYER                             │
│  [Tray] → [Vision Station] → [Bins: G/Y/R]            │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from roboticstoolbox import DHRobot, RevoluteDH
from spatialmath import SE3
import roboticstoolbox as rtb
from datetime import datetime
import json
from dataclasses import dataclass, asdict
from enum import Enum
import time

%matplotlib inline

## Task 4.1: Complete System Simulator

**Objective:** Build the complete simulation including robot motion, vision inspection, and sorting

In [ ]:
# Task 4.1: Import all components from previous sessions

# Copy your implementations from Sessions 1-3:
# - Robot model
# - Motion planner
# - Vision inspection system
# - Component generator

# For this demonstration, we'll use simplified versions

class OpenManipulatorX(DHRobot):
    def __init__(self):
        d1, a2, a3, a4 = 0.077, 0.130, 0.124, 0.126
        L = [
            RevoluteDH(d=d1, a=0, alpha=np.pi/2),
            RevoluteDH(d=0, a=a2, alpha=0),
            RevoluteDH(d=0, a=a3, alpha=0),
            RevoluteDH(d=0, a=a4, alpha=0),
        ]
        super().__init__(L, name="OpenManipulator-X")

class QualityLevel(Enum):
    GOOD = "GOOD"
    REWORK = "REWORK"
    REJECT = "REJECT"

@dataclass
class ProcessingRecord:
    """Data record for Industry 4.0 monitoring"""
    timestamp: str
    component_id: int
    cycle_time: float
    quality_decision: str
    bin_location: str
    dimensional_errors: list
    defects_detected: dict
    joint_trajectory: list = None  # Can store for analysis
    
class Industry40QualityCell:
    """
    Complete Industry 4.0 quality control cell with data logging
    """
    def __init__(self, robot):
        self.robot = robot
        self.processing_records = []
        
        # Performance metrics
        self.metrics = {
            'total_components': 0,
            'total_time': 0,
            'good_count': 0,
            'rework_count': 0,
            'reject_count': 0,
            'average_cycle_time': 0,
            'throughput': 0,  # components per hour
        }
    
    def process_component(self, component_id, quality_result, cycle_time, 
                          dim_errors, defects):
        """
        Process one component and log data
        """
        # Create record
        record = ProcessingRecord(
            timestamp=datetime.now().isoformat(),
            component_id=component_id,
            cycle_time=cycle_time,
            quality_decision=quality_result.value,
            bin_location=self._get_bin_location(quality_result),
            dimensional_errors=dim_errors.tolist() if isinstance(dim_errors, np.ndarray) else dim_errors,
            defects_detected=defects
        )
        
        self.processing_records.append(record)
        
        # Update metrics
        self.metrics['total_components'] += 1
        self.metrics['total_time'] += cycle_time
        
        if quality_result == QualityLevel.GOOD:
            self.metrics['good_count'] += 1
        elif quality_result == QualityLevel.REWORK:
            self.metrics['rework_count'] += 1
        else:
            self.metrics['reject_count'] += 1
        
        # Update average cycle time
        self.metrics['average_cycle_time'] = (
            self.metrics['total_time'] / self.metrics['total_components']
        )
        
        # Calculate throughput (components/hour)
        self.metrics['throughput'] = 3600 / self.metrics['average_cycle_time']
        
        return record
    
    def _get_bin_location(self, quality):
        bins = {
            QualityLevel.GOOD: "Green Bin (-150, 200, 100)",
            QualityLevel.REWORK: "Yellow Bin (0, 220, 100)",
            QualityLevel.REJECT: "Red Bin (150, 200, 100)"
        }
        return bins[quality]
    
    def export_data(self, filename='quality_data.json'):
        """
        Export processing data for Industry 4.0 analytics
        """
        data = {
            'metadata': {
                'robot': self.robot.name,
                'export_time': datetime.now().isoformat(),
                'total_records': len(self.processing_records)
            },
            'metrics': self.metrics,
            'processing_records': [asdict(r) for r in self.processing_records]
        }
        
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"Data exported to {filename}")
    
    def generate_dashboard(self):
        """
        Generate Industry 4.0 monitoring dashboard
        """
        fig = plt.figure(figsize=(16, 10))
        
        # Quality distribution (pie chart)
        ax1 = fig.add_subplot(2, 3, 1)
        sizes = [
            self.metrics['good_count'],
            self.metrics['rework_count'],
            self.metrics['reject_count']
        ]
        colors = ['#2ecc71', '#f39c12', '#e74c3c']
        labels = ['GOOD', 'REWORK', 'REJECT']
        ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        ax1.set_title('Quality Distribution')
        
        # Cycle time trend
        ax2 = fig.add_subplot(2, 3, 2)
        cycle_times = [r.cycle_time for r in self.processing_records]
        ax2.plot(cycle_times, marker='o', linestyle='-', markersize=4)
        ax2.axhline(y=np.mean(cycle_times), color='r', linestyle='--', 
                    label=f'Average: {np.mean(cycle_times):.2f}s')
        ax2.axhline(y=8.0, color='g', linestyle='--', alpha=0.5, label='Target: 8s')
        ax2.set_xlabel('Component Number')
        ax2.set_ylabel('Cycle Time (s)')
        ax2.set_title('Cycle Time Trend')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Throughput
        ax3 = fig.add_subplot(2, 3, 3)
        ax3.bar(['Throughput'], [self.metrics['throughput']], color='#3498db')
        ax3.set_ylabel('Components/Hour')
        ax3.set_title('Production Throughput')
        ax3.set_ylim([0, max(500, self.metrics['throughput'] * 1.2)])
        for i, v in enumerate([self.metrics['throughput']]):
            ax3.text(i, v + 10, f"{v:.0f}", ha='center', va='bottom', fontweight='bold')
        
        # Dimensional error distribution
        ax4 = fig.add_subplot(2, 3, 4)
        all_errors = []
        for r in self.processing_records:
            if r.dimensional_errors:
                all_errors.extend(r.dimensional_errors)
        if all_errors:
            ax4.hist(all_errors, bins=20, color='#9b59b6', alpha=0.7, edgecolor='black')
            ax4.axvline(x=0.2, color='orange', linestyle='--', label='GOOD threshold')
            ax4.axvline(x=0.3, color='red', linestyle='--', label='REWORK threshold')
            ax4.set_xlabel('Dimensional Error (mm)')
            ax4.set_ylabel('Frequency')
            ax4.set_title('Dimensional Error Distribution')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
        
        # Defect types
        ax5 = fig.add_subplot(2, 3, 5)
        defect_counts = {'crack': 0, 'scratch': 0, 'missing_component': 0}
        for r in self.processing_records:
            for defect, present in r.defects_detected.items():
                if present:
                    defect_counts[defect] += 1
        
        ax5.bar(defect_counts.keys(), defect_counts.values(), color='#e67e22')
        ax5.set_ylabel('Count')
        ax5.set_title('Defect Type Frequency')
        ax5.grid(True, alpha=0.3, axis='y')
        
        # Performance KPIs
        ax6 = fig.add_subplot(2, 3, 6)
        ax6.axis('off')
        kpi_text = f"""
        PERFORMANCE KPIs
        ═══════════════════════════════
        
        Total Processed:     {self.metrics['total_components']:>6d}
        Total Time:          {self.metrics['total_time']:>6.1f} s
        
        Average Cycle Time:  {self.metrics['average_cycle_time']:>6.2f} s
        Throughput:          {self.metrics['throughput']:>6.0f} pcs/hr
        
        Quality Yield:
          • GOOD:      {100*self.metrics['good_count']/max(1,self.metrics['total_components']):>6.1f}%
          • REWORK:    {100*self.metrics['rework_count']/max(1,self.metrics['total_components']):>6.1f}%
          • REJECT:    {100*self.metrics['reject_count']/max(1,self.metrics['total_components']):>6.1f}%
        
        Target Met:          {'✓ YES' if self.metrics['average_cycle_time'] < 8.0 else '✗ NO'}
        """
        ax6.text(0.1, 0.5, kpi_text, fontfamily='monospace', fontsize=11,
                verticalalignment='center')
        
        plt.suptitle('Industry 4.0 Quality Control Dashboard', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Create system
robot = OpenManipulatorX()
quality_cell = Industry40QualityCell(robot)
print("Industry 4.0 Quality Control Cell initialized")

## Task 4.2: Simulation Runner

Run complete simulation of production batch

In [ ]:
# Task 4.2: Production simulation

def simulate_production_batch(n_components=30):
    """
    Simulate processing a batch of components
    """
    print(f"Starting production simulation: {n_components} components\n")
    
    for i in range(n_components):
        # Simulate quality inspection
        # In real system, this would come from actual vision system
        quality = np.random.choice(
            [QualityLevel.GOOD, QualityLevel.REWORK, QualityLevel.REJECT],
            p=[0.85, 0.10, 0.05]
        )
        
        # Simulate dimensional errors
        if quality == QualityLevel.GOOD:
            dim_errors = np.random.uniform(0, 0.15, 3)
        elif quality == QualityLevel.REWORK:
            dim_errors = np.random.uniform(0.15, 0.25, 3)
        else:
            dim_errors = np.random.uniform(0.25, 0.50, 3)
        
        # Simulate defects
        defects = {
            'crack': quality == QualityLevel.REJECT and np.random.random() < 0.5,
            'scratch': quality != QualityLevel.GOOD and np.random.random() < 0.4,
            'missing_component': quality == QualityLevel.REJECT and np.random.random() < 0.2
        }
        
        # Simulate cycle time (varies by position and bin)
        base_time = 5.5  # seconds
        position_factor = 1.0 + 0.1 * (i % 9) / 9  # Components further away take longer
        bin_factor = {'GOOD': 1.0, 'REWORK': 1.05, 'REJECT': 1.08}
        cycle_time = base_time * position_factor * bin_factor[quality.value]
        cycle_time += np.random.normal(0, 0.2)  # Add some randomness
        
        # Process component
        record = quality_cell.process_component(
            component_id=i,
            quality_result=quality,
            cycle_time=cycle_time,
            dim_errors=dim_errors,
            defects=defects
        )
        
        # Progress indicator
        if (i + 1) % 10 == 0:
            print(f"Processed {i+1}/{n_components} components... "
                  f"Avg cycle time: {quality_cell.metrics['average_cycle_time']:.2f}s")
    
    print(f"\nSimulation complete!")
    print(f"Total time: {quality_cell.metrics['total_time']:.1f} seconds")
    print(f"Throughput: {quality_cell.metrics['throughput']:.0f} components/hour")

# Run simulation
simulate_production_batch(50)

# Generate dashboard
quality_cell.generate_dashboard()

## Task 4.3: Cycle Time Optimization

**Objective:** Analyze and optimize the system to meet the <8 second cycle time target

**Optimization strategies:**
1. Trajectory optimization (faster motion, optimized paths)
2. Parallel operations (inspect while moving)
3. Workspace reconfiguration (optimize bin positions)
4. Motion blending (smooth transitions between waypoints)

In [ ]:
# Task 4.3: Optimization analysis

class CycleTimeOptimizer:
    """
    Analyze and optimize cycle time
    """
    def __init__(self):
        # Baseline timings (seconds)
        self.baseline = {
            'home_to_tray': 1.5,
            'approach_to_pick': 0.5,
            'pick_operation': 0.3,
            'tray_to_vision': 1.2,
            'vision_scan': 1.5,
            'vision_to_bin': 1.3,
            'approach_to_place': 0.5,
            'place_operation': 0.3,
            'bin_to_home': 1.4,
        }
        
        self.baseline_total = sum(self.baseline.values())
    
    def analyze_baseline(self):
        """
        Analyze baseline cycle time
        """
        print("BASELINE CYCLE TIME ANALYSIS")
        print("="*50)
        
        for operation, time in self.baseline.items():
            percentage = 100 * time / self.baseline_total
            bar = '█' * int(percentage / 2)
            print(f"{operation:20s}: {time:4.1f}s [{bar:25s}] {percentage:4.1f}%")
        
        print("-" * 50)
        print(f"{'TOTAL':20s}: {self.baseline_total:4.1f}s")
        print(f"{'Target':20s}:  8.0s")
        print(f"{'Reduction needed':20s}: {self.baseline_total - 8.0:4.1f}s ({100*(self.baseline_total-8.0)/self.baseline_total:.1f}%)")
        print("=" * 50)
    
    def propose_optimizations(self):
        """
        Propose optimization strategies
        """
        optimizations = [
            {
                'name': 'Increase motion speed by 20%',
                'affected_operations': ['home_to_tray', 'tray_to_vision', 'vision_to_bin', 'bin_to_home'],
                'reduction_factor': 0.20,
                'risk': 'Medium (may affect accuracy, increase wear)'
            },
            {
                'name': 'Reduce vision scan time (faster scanning)',
                'affected_operations': ['vision_scan'],
                'reduction_factor': 0.30,
                'risk': 'High (may reduce inspection quality)'
            },
            {
                'name': 'Optimize pick/place motions',
                'affected_operations': ['approach_to_pick', 'pick_operation', 'approach_to_place', 'place_operation'],
                'reduction_factor': 0.15,
                'risk': 'Low'
            },
            {
                'name': 'Relocate bins closer (workspace optimization)',
                'affected_operations': ['vision_to_bin'],
                'reduction_factor': 0.25,
                'risk': 'Low (requires workspace redesign)'
            },
        ]
        
        print("\nPROPOSED OPTIMIZATIONS")
        print("=" * 80)
        
        for i, opt in enumerate(optimizations, 1):
            # Calculate new cycle time
            new_times = self.baseline.copy()
            for op in opt['affected_operations']:
                new_times[op] *= (1 - opt['reduction_factor'])
            
            new_total = sum(new_times.values())
            time_saved = self.baseline_total - new_total
            
            print(f"\nOption {i}: {opt['name']}")
            print(f"  Time saved:    {time_saved:.2f}s ({100*time_saved/self.baseline_total:.1f}%)")
            print(f"  New cycle time: {new_total:.2f}s")
            print(f"  Target met:    {'✓ YES' if new_total < 8.0 else '✗ NO'}")
            print(f"  Risk level:    {opt['risk']}")
        
        # Combined optimization
        print("\n" + "-" * 80)
        print("\nCOMBINED OPTIMIZATION (All strategies)")
        combined_times = self.baseline.copy()
        for opt in optimizations:
            for op in opt['affected_operations']:
                combined_times[op] *= (1 - opt['reduction_factor'])
        
        combined_total = sum(combined_times.values())
        total_saved = self.baseline_total - combined_total
        
        print(f"  Original cycle time: {self.baseline_total:.2f}s")
        print(f"  Optimized cycle time: {combined_total:.2f}s")
        print(f"  Total time saved: {total_saved:.2f}s ({100*total_saved/self.baseline_total:.1f}%)")
        print(f"  Target met: {'✓ YES' if combined_total < 8.0 else '✗ NO'}")
        print("=" * 80)
        
        return optimizations, combined_times

# Run optimization analysis
optimizer = CycleTimeOptimizer()
optimizer.analyze_baseline()
optimizations, optimized_times = optimizer.propose_optimizations()

## Task 4.4: Industry 4.0 Data Analytics

**Objective:** Implement data analytics for continuous improvement

Industry 4.0 features:
- Real-time monitoring
- Predictive maintenance
- Quality trends analysis
- OEE (Overall Equipment Effectiveness) calculation

In [ ]:
# Task 4.4: Industry 4.0 analytics

class Industry40Analytics:
    """
    Advanced analytics for Industry 4.0 quality control
    """
    def __init__(self, quality_cell):
        self.cell = quality_cell
    
    def calculate_oee(self, ideal_cycle_time=5.0, planned_production_time=3600):
        """
        Calculate Overall Equipment Effectiveness (OEE)
        OEE = Availability × Performance × Quality
        """
        if self.cell.metrics['total_components'] == 0:
            return {'availability': 0, 'performance': 0, 'quality': 0, 'oee': 0}
        
        # Availability (assuming no downtime in simulation)
        actual_production_time = self.cell.metrics['total_time']
        availability = min(1.0, actual_production_time / planned_production_time)
        
        # Performance
        actual_cycle_time = self.cell.metrics['average_cycle_time']
        performance = min(1.0, ideal_cycle_time / actual_cycle_time)
        
        # Quality (ratio of good parts)
        good_parts = self.cell.metrics['good_count']
        total_parts = self.cell.metrics['total_components']
        quality = good_parts / total_parts if total_parts > 0 else 0
        
        # OEE
        oee = availability * performance * quality
        
        return {
            'availability': availability,
            'performance': performance,
            'quality': quality,
            'oee': oee
        }
    
    def quality_trend_analysis(self):
        """
        Analyze quality trends over time
        """
        if not self.cell.processing_records:
            return
        
        # Group by batches of 10
        batch_size = 10
        n_batches = len(self.cell.processing_records) // batch_size
        
        batch_quality = []
        batch_cycle_times = []
        
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = start_idx + batch_size
            batch = self.cell.processing_records[start_idx:end_idx]
            
            # Quality rate
            good_count = sum(1 for r in batch if r.quality_decision == 'GOOD')
            quality_rate = good_count / len(batch)
            batch_quality.append(quality_rate)
            
            # Average cycle time
            avg_time = np.mean([r.cycle_time for r in batch])
            batch_cycle_times.append(avg_time)
        
        # Plot trends
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Quality trend
        axes[0].plot(range(1, n_batches + 1), [q * 100 for q in batch_quality], 
                     marker='o', linewidth=2, markersize=8)
        axes[0].axhline(y=85, color='g', linestyle='--', alpha=0.5, label='Target: 85%')
        axes[0].set_xlabel('Batch Number (10 components each)')
        axes[0].set_ylabel('Quality Rate (%)')
        axes[0].set_title('Quality Trend Analysis')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        axes[0].set_ylim([0, 105])
        
        # Cycle time trend
        axes[1].plot(range(1, n_batches + 1), batch_cycle_times, 
                     marker='s', linewidth=2, markersize=8, color='orange')
        axes[1].axhline(y=8.0, color='r', linestyle='--', alpha=0.5, label='Target: 8s')
        axes[1].set_xlabel('Batch Number (10 components each)')
        axes[1].set_ylabel('Average Cycle Time (s)')
        axes[1].set_title('Cycle Time Trend Analysis')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def generate_report(self):
        """
        Generate comprehensive Industry 4.0 report
        """
        oee_metrics = self.calculate_oee()
        
        print("\n" + "═" * 70)
        print("INDUSTRY 4.0 QUALITY CONTROL - PERFORMANCE REPORT")
        print("═" * 70)
        
        print("\n📊 PRODUCTION SUMMARY")
        print("-" * 70)
        print(f"Total Components Processed:  {self.cell.metrics['total_components']}")
        print(f"Total Production Time:       {self.cell.metrics['total_time']:.1f} seconds")
        print(f"Average Cycle Time:          {self.cell.metrics['average_cycle_time']:.2f} seconds")
        print(f"Throughput:                  {self.cell.metrics['throughput']:.0f} components/hour")
        
        print("\n🎯 QUALITY METRICS")
        print("-" * 70)
        total = self.cell.metrics['total_components']
        print(f"GOOD components:      {self.cell.metrics['good_count']:4d} ({100*self.cell.metrics['good_count']/max(1,total):5.1f}%)")
        print(f"REWORK components:    {self.cell.metrics['rework_count']:4d} ({100*self.cell.metrics['rework_count']/max(1,total):5.1f}%)")
        print(f"REJECT components:    {self.cell.metrics['reject_count']:4d} ({100*self.cell.metrics['reject_count']/max(1,total):5.1f}%)")
        
        print("\n⚙️  OEE (OVERALL EQUIPMENT EFFECTIVENESS)")
        print("-" * 70)
        print(f"Availability:         {oee_metrics['availability']*100:5.1f}%")
        print(f"Performance:          {oee_metrics['performance']*100:5.1f}%")
        print(f"Quality:              {oee_metrics['quality']*100:5.1f}%")
        print(f"")
        print(f"OEE Score:            {oee_metrics['oee']*100:5.1f}%")
        
        # OEE Benchmarks
        if oee_metrics['oee'] >= 0.85:
            oee_rating = "🏆 WORLD CLASS"
        elif oee_metrics['oee'] >= 0.60:
            oee_rating = "✓ ACCEPTABLE"
        else:
            oee_rating = "✗ NEEDS IMPROVEMENT"
        print(f"OEE Rating:           {oee_rating}")
        
        print("\n🎓 RECOMMENDATIONS")
        print("-" * 70)
        
        recommendations = []
        if self.cell.metrics['average_cycle_time'] > 8.0:
            recommendations.append("⚠ Cycle time exceeds target. Consider optimization strategies.")
        if oee_metrics['performance'] < 0.80:
            recommendations.append("⚠ Performance below target. Review motion planning and trajectory optimization.")
        if oee_metrics['quality'] < 0.85:
            recommendations.append("⚠ Quality rate below target. Review inspection parameters and tolerances.")
        if self.cell.metrics['reject_count'] / max(1, total) > 0.10:
            recommendations.append("⚠ High reject rate. Investigate root causes in supplier quality.")
        
        if not recommendations:
            recommendations.append("✓ System operating within target parameters. Continue monitoring.")
        
        for rec in recommendations:
            print(f"  {rec}")
        
        print("\n" + "═" * 70)

# Run analytics
analytics = Industry40Analytics(quality_cell)
analytics.generate_report()
analytics.quality_trend_analysis()

## Task 4.5: Export Data and Final Report

In [ ]:
# Task 4.5: Export production data

# Export JSON data for further analysis
quality_cell.export_data('quality_control_data.json')

# Generate final dashboard
quality_cell.generate_dashboard()

print("\n✅ All data exported successfully!")
print("\nFiles generated:")
print("  - quality_control_data.json (raw data for analytics)")
print("  - Dashboard visualizations (displayed above)")

---

## Session 4 Deliverables

**By the end of Session 4, you should have:**

✅ Complete integrated simulation of the quality control cell  
✅ Industry 4.0 monitoring dashboard with real-time KPIs  
✅ Cycle time optimization analysis  
✅ OEE calculation and performance metrics  
✅ Quality trend analysis  
✅ Data export for further analytics  
✅ Comprehensive performance report  

---

## Final Project Deliverables

### 1. Technical Report (PDF)

Your report must include:

**Section 1: Introduction (5%)**
- Problem statement
- Industry 4.0 context
- Project objectives

**Section 2: Robot Modeling (15%)**
- DH parameters (standard and modified)
- Elementary transformation derivation
- Forward kinematics validation
- Workspace analysis

**Section 3: Motion Planning (20%)**
- Inverse kinematics (analytical and numerical)
- Trajectory generation methods
- Complete pick-and-place sequence
- Cycle time analysis

**Section 4: Quality Inspection (15%)**
- Vision system simulation
- Classification algorithm
- Jacobian-based velocity control
- Inspection performance analysis

**Section 5: System Integration (15%)**
- Complete system architecture
- Simulation results
- Performance optimization
- Industry 4.0 features

**Section 6: Results and Discussion (20%)**
- Performance metrics (cycle time, throughput, OEE)
- Quality statistics
- Optimization results
- Challenges and solutions

**Section 7: Conclusion (5%)**
- Summary of achievements
- Future improvements
- Lessons learned

**Appendices (5%)**
- Hand-written calculations
- Additional plots and data
- Code listings

### 2. Python Implementation

Submit all Jupyter notebooks:
- Session 1: Robot modeling
- Session 2: Motion planning
- Session 3: Quality inspection
- Session 4: System integration

### 3. Demonstration Video (10 minutes)

Record a video showing:
- System overview and architecture
- Live simulation run
- Dashboard and analytics
- Key results and conclusions

### 4. Presentation (15 minutes + 5 min Q&A)

Prepare slides covering:
- Problem and solution overview
- Key technical approaches
- Results and achievements
- Challenges overcome
- Future work

---

## Grading Rubric

| Component | Weight | Criteria |
|-----------|--------|----------|
| Technical Report | 60% | Accuracy, completeness, clarity, analysis depth |
| Python Implementation | 30% | Code quality, functionality, documentation |
| Demonstration | 10% | Presentation quality, Q&A responses |

**Bonus points (+10%) for:**
- Exceptionally innovative solutions
- Advanced Industry 4.0 features
- Real-time visualization
- Comprehensive optimization analysis

---

## Congratulations!

You have completed the Industry 4.0 Quality Control SAE project. You've learned to:

✓ Model robotic manipulators using multiple kinematic conventions  
✓ Plan and execute complex pick-and-place operations  
✓ Integrate vision-based quality inspection  
✓ Optimize system performance  
✓ Implement Industry 4.0 monitoring and analytics  

**Good luck with your demonstration!** 🎓🤖